In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.decomposition import PCA, FastICA
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from scipy.stats import randint as sp_randint

import random
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from mlxtend.regressor import StackingRegressor

import xgboost as xgb

## Reading and preprocessing data

In [2]:
x_train = pd.read_csv('train.csv', index_col='ID')
x_test = pd.read_csv('test.csv', index_col='ID')
y_train = x_train['y']
x_train.drop('y', axis=1, inplace=True)

le = LabelEncoder()

In [3]:
def str_to_int(df, col_name):
    if df[col_name].dtype != 'int64':
        le.fit(df[col_name].values)
        df[col_name] = le.transform(df[col_name])

for col_name in x_train.columns:
    str_to_int(x_train, col_name)
    str_to_int(x_test, col_name)

### Feature selection and generation

In [4]:
selector = SelectKBest(k=75, score_func=f_regression).fit(x_train, y_train)
x_train_new = selector.transform(x_train)
x_test_new = selector.transform(x_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:279: RuntimeWarning: invalid value encountered in true_divide
  corr /= row_norms(X.T)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [5]:
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)

# ICA
ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(x_train)
ica2_results_test = ica.transform(x_test)

x_train_new = pd.DataFrame(x_train_new)
x_test_new = pd.DataFrame(x_test_new)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    x_train_new['pca_' + str(i)] = pca2_results_train[:,i-1]
    x_test_new['pca_' + str(i)] = pca2_results_test[:, i-1]
    
    x_train_new['ica_' + str(i)] = ica2_results_train[:,i-1]
    x_test_new['ica_' + str(i)] = ica2_results_test[:, i-1]
    
y_mean = np.mean(y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [6]:
X_train_pca, X_val_pca, y_train_pca, y_val_pca = train_test_split(x_train_new, y_train, 
                                                                  random_state=42, train_size=.95)

### Raw data
#### run that cell before you run cell *Feature selection and generation*

In [ ]:
X_train_unf, X_val_unf, y_train_unf, y_val_unf = train_test_split(
    x_train, y_train, random_state=42, train_size=.95)

In [ ]:
ada_boost = AdaBoostRegressor(base_estimator=None, n_estimators=10, learning_rate=0.1, 
                              loss='square', random_state=None)
ada_boost.fit(X_train_unf, y_train_unf)
y_pred = ada_boost.predict(X_val_unf)
print(r2_score(y_val_unf, y_pred))
y_pred = ada_boost.predict(x_test)

## GridSearchCV: find the best hyper-params

In [ ]:
parameters = {'n_estimators':range(50,60,1), 
              'learning_rate':np.arange(0.001, 0.01, 0.001)}
grid = GridSearchCV(estimator=ada_boost, param_grid=parameters, n_jobs=-1, scoring='r2')
grid.fit(X_train_unf, y_train_unf)
grid.best_params_, grid.best_score_

In [ ]:
ada_boost = AdaBoostRegressor(base_estimator=None, 
                              n_estimators=59,
                              learning_rate=0.002,
                              loss='square')

In [ ]:
ada_boost.fit(X_train_unf, y_train_unf)
y_pred = ada_boost.predict(X_val_unf)
print(r2_score(y_val_unf, y_pred))
y_pred = ada_boost.predict(x_test)

In [ ]:
cross_val = cross_val_score(estimator=ada_boost, X=x_train, y=y_train, n_jobs=-1, cv=4, verbose=1)

In [ ]:
np.mean(cross_val)

### After selectKBest

In [ ]:
X_train_sel, X_val_sel, y_train_sel, y_val_sel = train_test_split(
    x_train_new, y_train, random_state=42, train_size=.95)

In [ ]:
ada_boost = AdaBoostRegressor(base_estimator=None, 
                              n_estimators=59,
                              learning_rate=0.002,
                              loss='square')
ada_boost.fit(X_train_sel, y_train_sel)
y_pred = ada_boost.predict(X_val_sel)
print(r2_score(y_val_sel, y_pred))
y_pred = ada_boost.predict(x_test_new)

In [ ]:
cross_val_sel = cross_val_score(estimator=ada_boost, X=x_train_new, y=y_train, n_jobs=-1, cv=4, verbose=1)

In [ ]:
np.mean(cross_val_sel)

### After selectKBest+PCA/ICA

In [7]:
ada_boost = AdaBoostRegressor(base_estimator=None, 
                              n_estimators=59,
                              learning_rate=0.002,
                              loss='square')
#X_train_pca, X_val_pca, y_train_pca, y_val_pca
ada_boost.fit(X_train_pca, y_train_pca)
y_pred = ada_boost.predict(X_val_pca)
print(r2_score(y_val_pca, y_pred))
y_pred = ada_boost.predict(x_test_new)

0.641689572945


In [8]:
cross_val_pca = cross_val_score(estimator=ada_boost, X=x_train_new, y=y_train, n_jobs=-1, cv=4, verbose=1)

[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    2.4s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.4s finished


In [10]:
np.mean(cross_val_pca)

0.56194309124864195

In [ ]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[svr_lin, lr, ridge], 
                           meta_regressor=svr_rbf)

# Training the stacking classifier

stregr.fit(X_train, y_train)
y_pred = stregr.predict(X_val)
r2_score(y_val, y_pred)

In [ ]:
y_pred = stregr.predict(x_test_new)

### Training model

In [ ]:
xgb_params = {
    'n_trees': 374, 
    'eta': 0.15000000000000002,
    'gamma': 0.6000000000000001,
    'max_depth': 4,
    'subsample': 0.9500000000000001,
    'min_child_weight': 2.0,
    'colsample_bytree': 0.9500000000000001,
    'objective': 'reg:linear',
    'nthread': 8,
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# xgb_params = {'colsample_bytree': 0.65, 'eta': 0.30000000000000004, 'eval_metric': 'rmse', 'gamma': 0.9500000000000001, 'max_depth': 8, 'min_child_weight': 1.0, 'n_estimators': 110.0, 'nthread': 6, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.9}
# xgb_params_from_iamtodor = {'colsample_bytree': 0.9500000000000001, 'eta': 0.15000000000000002, 'eval_metric': 'rmse', 'gamma': 0.6000000000000001, 'max_depth': 4, 'min_child_weight': 2.0, 'n_estimators': 169.0, 'nthread': 8, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.9500000000000001}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(x_test_new)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=1500,
                   early_stopping_rounds=150,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

## Tuning hyperparameters

In [ ]:
def score(params):
    print("Training with params : ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid)
    score = r2_score(y_val, predictions)
    print("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


In [ ]:
def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 1),
             'eta' : hp.quniform('eta', 0.0001, 0.5, 0.025),
             'max_depth' : sample(scope.int(hp.quniform('max_depth', 0, 10, 1))),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'eval_metric': 'rmse',
             'objective': 'reg:linear',
             'nthread' : 8,
             'silent' : 1
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)

    print(best)

In [ ]:
trials = Trials()

optimize(trials)

## Generate submission file

In [12]:
#y_pred = model.predict(dtest)
output = pd.DataFrame({'id': x_test.index, 'y': y_pred})
output.to_csv('submition.csv', index=False)